# License
    IPython notebook for simulating the shallow water equations with OpenCL
    Copyright (C) 2015 Andre.Brodtkorb@ifi.uio.no

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [1]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Lets have opencl ipython integration enabled
%load_ext pyopencl.ipython_ext

#Import packages we need
import numpy as np
import pyopencl as cl
import os
import time
from matplotlib import animation, rc
from matplotlib import pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

In [2]:
#Setup easier to use compilation of OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_CTX"] = "0"
os.environ["CUDA_CACHE_DISABLE"] = "1"

In [3]:
#Create OpenCL context
cl_ctx = cl.create_some_context()

print("Using ", cl_ctx.devices[0].name)

#Create an OpenCL command queue
cl_queue = cl.CommandQueue(cl_ctx)

Using  GeForce GTX 780


# Shallow Water Equations in 2D
The homogeneous Shallow Water Equations in 2D can be written
$$
\begin{align}
\frac{\partial h}{\partial t} + \frac{\partial hu}{\partial x} + \frac{\partial hv}{\partial y} &= 0\\
\frac{\partial hu}{\partial t} + \frac{\partial [hu^2 + \tfrac{1}{2}gh^2]}{\partial x} + \frac{\partial huv}{\partial y} &= 0\\
\frac{\partial hv}{\partial t} + \frac{\partial huv}{\partial x} + \frac{\partial [hv^2 + \tfrac{1}{2}gh^2]}{\partial y} &= 0
\end{align}
$$
or using vectorized notation:
$$
\begin{align}
\left[
\begin{array}{c}
h\\
hu\\
hv
\end{array}
\right]_t
+
\left[
\begin{array}{c}
hu\\
hu^2+\tfrac{1}{2}gh^2\\
huv
\end{array}
\right]_x
+
\left[
\begin{array}{c}
hv\\
huv\\
hv^2+\tfrac{1}{2}gh^2
\end{array}
\right]_y
&=
\left[
\begin{array}{c}
0 \\
0 \\
0
\end{array}
\right]
\end{align} 
$$
or even shorter
$$
\frac{\partial Q}{\partial t} + \frac{\partial F(Q)}{\partial x} + \frac{\partial G(Q)}{\partial y} = 0
$$

We can write the classical Lax-Friedrichs numerical scheme that we will use as
$$
\begin{align}
Q_{i, j}^{n+1} &=& \frac{1}{4} \left[ Q_{i+1, j}^{n} + Q_{i-1, j}^{n} + Q_{i, j+1}^{n} + Q_{i, j-1}^{n} \right]\\
&&- \frac{\Delta t}{2\Delta x} \left[F(Q_{i+1, j}^{n}) - F(Q_{i-1, j}^{n}) \right]\\
&&- \frac{\Delta t}{2\Delta y} \left[G(Q_{i, j+1}^{n}) - G(Q_{i, j-1}^{n}) \right]
\end{align}
$$
When implementing, we simply use the above equation for each of the three components of our equation set.

This discretization is unstable if the following CFL condition is not met
$$
\Delta t \lt \text{min}_{i, j}\left[\frac{1}{\Delta x} |u^n_{i, j} \pm \sqrt{gh^n_{i,j}}|, \frac{1}{\Delta x} |v^n_{i, j} \pm \sqrt{gh^n_{i,j}}|\right]
$$

In [4]:
%%cl_kernel 

float3 F(const float3 Q, const float g) {
    float3 F;

    F.x = Q.y;                              //hu
    F.y = Q.y*Q.y / Q.x + 0.5f*g*Q.x*Q.x;   //hu*hu/h + 0.5f*g*h*h;
    F.z = Q.y*Q.z / Q.x;                    //hu*hv/h;

    return F;
}

float3 G(const float3 Q, const float g) {
    float3 G;

    G.x = Q.z;                              //hv
    G.y = Q.y*Q.z / Q.x;                    //hu*hv/h;
    G.z = Q.z*Q.z / Q.x + 0.5f*g*Q.x*Q.x;   //hv*hv/h + 0.5f*g*h*h;

    return G;
}

__kernel void swe_2D(
        __global float* h1, __global float* hu1, __global float* hv1,
        __global const float *h0, __global const float *hu0, __global const float *hv0,
        float g,
        float dt, float dx, float dy) {

    //Get total number of cells
    int nx = get_global_size(0); 
    int ny = get_global_size(1);

    //Get position in grid
    int i = get_global_id(0); 
    int j = get_global_id(1);

    //Internal cells
    if (i > 0 && i < nx-1 && j > 0 && j <ny-1) {
        //Calculate the four indices of our neighboring cells
        int i = get_global_id(0); 
        int j = get_global_id(1);

        int center = j*nx + i;
        int north = (j+1)*nx + i;
        int south = (j-1)*nx + i;
        int east = j*nx + i+1;
        int west = j*nx + i-1;

        const float3 Q_east = (float3)(h0[east], hu0[east], hv0[east]);
        const float3 Q_west = (float3)(h0[west], hu0[west], hv0[west]);
        const float3 Q_north = (float3)(h0[north], hu0[north], hv0[north]);
        const float3 Q_south = (float3)(h0[south], hu0[south], hv0[south]);

        const float3 F_east = F(Q_east, g);
        const float3 F_west = F(Q_west, g);
        const float3 G_north = G(Q_north, g);
        const float3 G_south = G(Q_south, g);

        float3 Q1 = 0.25f*(Q_east + Q_west + Q_north + Q_south)
            - dt/(2.0f*dx)*(F_east - F_west)
            - dt/(2.0f*dy)*(G_north - G_south);

        h1[center] = Q1.x;
        hu1[center] = Q1.y;
        hv1[center] = Q1.z;
    }
}

In [5]:
%%cl_kernel
__kernel void swe_2D_bc(__global float* h, __global float* hu, __global float* hv) {
    //Get total number of cells
    int nx = get_global_size(0); 
    int ny = get_global_size(1);

    //Get position in grid
    int i = get_global_id(0); 
    int j = get_global_id(1); 

    //Calculate the four indices of our neighboring cells
    int center = j*nx + i;
    int north = (j+1)*nx + i;
    int south = (j-1)*nx + i;
    int east = j*nx + i+1;
    int west = j*nx + i-1;

    if (i == 0) {
        h[center] = h[east];
        hu[center] = -hu[east];
        hv[center] = hv[east];
    }
    else if (i == nx-1) {
        h[center] = h[west];
        hu[center] = -hu[west];
        hv[center] = hv[west];
    }
    else if (j == 0) {
        h[center] = h[north];
        hu[center] = hu[north];
        hv[center] = -hv[north];
    }
    else if (j == ny-1) {
        h[center] = h[south];
        hu[center] = hu[south];
        hv[center] = -hv[south];
    }
}

In [10]:
"""
Class that holds data for the heat equation in OpenCL
"""
class SWEDataCL:
    """
    Uploads initial data to the CL device
    """
    def __init__(self, h0, hu0, hv0):
        #Make sure that the data is single precision floating point
        assert(np.issubdtype(h0.dtype, np.float32))
        assert(np.issubdtype(hu0.dtype, np.float32))
        assert(np.issubdtype(hv0.dtype, np.float32))

        assert(not np.isfortran(h0))
        assert(not np.isfortran(hu0))
        assert(not np.isfortran(hv0))

        assert(h0.shape == hu0.shape)
        assert(h0.shape == hv0.shape)

        self.nx = h0.shape[1]
        self.ny = h0.shape[0]

        #Upload data to the device
        mf = cl.mem_flags
        self.h0 = cl.Buffer(cl_ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=h0)
        self.hu0 = cl.Buffer(cl_ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=hu0)
        self.hv0 = cl.Buffer(cl_ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=hv0)

        self.h1 = cl.Buffer(cl_ctx, mf.READ_WRITE, h0.nbytes)
        self.hu1 = cl.Buffer(cl_ctx, mf.READ_WRITE, h0.nbytes)
        self.hv1 = cl.Buffer(cl_ctx, mf.READ_WRITE, h0.nbytes)
        
    """
    Enables downloading data from CL device to Python
    """
    def download(self):
        #Allocate data on the host for result
        h1 = np.empty((self.ny, self.nx), dtype=np.float32)
        hu1 = np.empty((self.ny, self.nx), dtype=np.float32)
        hv1 = np.empty((self.ny, self.nx), dtype=np.float32)
        
        #Copy data from device to host
        cl.enqueue_copy(cl_queue, h1, self.h0)
        cl.enqueue_copy(cl_queue, hu1, self.hu0)
        cl.enqueue_copy(cl_queue, hv1, self.hv0)
        
        #Return
        return h1, hu1, hv1;

In [7]:
"""
Computes a solution to the shallow water equations using an explicit finite difference scheme with OpenCL
"""
def opencl_swe(cl_data, g, dx, dy, dt, nt):

    #Loop through all the timesteps
    for i in range(0, nt):
        #Execute program on device
        swe_2D(cl_queue, (cl_data.nx, cl_data.ny), None, \
               cl_data.h1, cl_data.hu1, cl_data.hv1, \
               cl_data.h0, cl_data.hu0, cl_data.hv0,
               np.float32(g), np.float32(dt), np.float32(dx), np.float32(dy))
        swe_2D_bc(cl_queue, (cl_data.nx, cl_data.ny), None, cl_data.h1, cl_data.hu1, cl_data.hv1)
        
        #Swap variables
        cl_data.h0, cl_data.h1 = cl_data.h1, cl_data.h0
        cl_data.hu0, cl_data.hu1 = cl_data.hu1, cl_data.hu0
        cl_data.hv0, cl_data.hv1 = cl_data.hv1, cl_data.hv0

In [8]:
def circular_dambreak_initial_conditions(nx, ny):
    dx = 100.0 / float(nx)
    dy = 100.0 / float(ny)
    dt = 0.05*min(dx, dy) #Estimate of dt that will not violate the CFL condition

    h0 = np.ones((ny, nx), dtype=np.float32);
    hu0 = np.zeros((ny, nx), dtype=np.float32);
    hv0 = np.zeros((ny, nx), dtype=np.float32);

    for j in range(ny):
        for i in range(nx):
            x = (i - nx/2.0) * dx
            y = (j - ny/2.0) * dy
            if (np.sqrt(x**2 + y**2) < 10*min(dx, dy)):
                h0[j, i] = 10.0

    cl_data = SWEDataCL(h0, hu0, hv0)
    return cl_data, dx, dy, dt

In [18]:
#Create test input data
cl_data, dx, dy, dt = circular_dambreak_initial_conditions(64, 64)
g = 9.80665

#Plot initial conditions
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
fig.suptitle("Wave equation 2D", fontsize=18)

max_x = cl_data.nx*dx
max_y = cl_data.ny*dy

y, x = np.mgrid[0:max_y:dy, 0:max_x:dx]
surf_args=dict(cmap=cm.coolwarm, shade=True, vmin=0.0, vmax=1.0, cstride=1, rstride=1)


def animate(i):
    timesteps_per_plot=10

    #Simulate
    if (i>0):
        opencl_swe(cl_data, g, dx, dy, dt, timesteps_per_plot)

    #Download data
    h1, hu1, hv1 = cl_data.download()

    #Plot
    ax.clear()
    ax.plot_surface(x, y, h1, **surf_args)
    ax.set_zlim(-5.0, 5.0)

    
anim = animation.FuncAnimation(fig, animate, range(50), interval=100)
plt.close(anim._fig)
anim